In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [12]:
pd.set_option('display.max_rows', 100)
pd.options.display.max_colwidth = 100

In [13]:
def filter_price(price):
    '''this function receives product price. product price also contains currency symbol which is sliced first. then 
       it removes comma. and through exception handling price is returned
    '''
    sliced_price = price[1:]
    sp_replace = sliced_price.replace(',', '')
    try:
        return (int(sp_replace))
    except:
        return np.NaN

In [14]:
# base_url = 'https://www.flipkart.com/laptops/~cs-a13vamjvkr/pr?sid=6bo%2Cb5g&collection-tab-name=Performance&ctx=eyJjYXJkQ29udGV4dCI6eyJhdHRyaWJ1dGVzIjp7InRpdGxlIjp7Im11bHRpVmFsdWVkQXR0cmlidXRlIjp7ImtleSI6InRpdGxlIiwiaW5mZXJlbmNlVHlwZSI6IlRJVExFIiwidmFsdWVzIjpbIlBlcmZvcm1hbmNlIExhcHRvcHMiXSwidmFsdWVUeXBlIjoiTVVMVElfVkFMVUVEIn19fX19&wid=28.productCard.PMU_V2_5&page='
base_url = 'https://www.flipkart.com/laptops/pr?sid=6bo%2Cb5g&page='

num_pages = 64

df = pd.DataFrame(columns=['Product', '₹ Current Price', '₹ Price Before', 'Discount'])

for page in range(1, num_pages):
    url = base_url + str(page)
    
    response = requests.get(url)

    soup = BeautifulSoup(response.content, 'html.parser')
    
    laptop_list = soup.find_all('div', {'class':'_2kHMtA'})
    
    for each_laptop in laptop_list:
        title = each_laptop.find('div', {'class': '_4rR01T'}).text
        discounted_price = each_laptop.find('div', {'class': '_30jeq3 _1_WHN1'}).text
        dp_filtered = filter_price(discounted_price)
        original_price = each_laptop.find('div', {'class': '_3I9_wc _27UcVY'}).text if each_laptop.find('div', {'class': '_3I9_wc _27UcVY'}) else 'NaN'
        op_filtered = filter_price(original_price)
        discount = each_laptop.find('div', {'class': '_3Ay6Sb'}).text if each_laptop.find('div', {'class': '_3Ay6Sb'}) else np.NaN
        
        df = pd.concat([df, pd.DataFrame({'Product':title, '₹ Current Price':dp_filtered, '₹ Price Before':op_filtered, 'Discount':discount, 'Rating':np.NaN}, index=[0])], ignore_index=True)
#         df = df.append({'Product':title, '₹ Current Price':dp_filtered, '₹ Price Before':op_filtered, 'Discount':discount}, ignore_index=True)  ==> soon to be deprecated

In [15]:
df.to_csv('flipkart_laptop.csv', index=False)

In [16]:
new_df = pd.read_csv('flipkart_laptop.csv')
new_df

,Product,₹ Current Price,₹ Price Before,Discount,Rating
0,Infinix INBook Y1 Plus Intel Core i3 10th Gen - (8 GB/256 GB SSD/Windows 11 Home) XL28 Thin and ...,27990,44990.0,37% off,NaN
1,ASUS Chromebook Flip Touch Celeron Dual Core - (4 GB/32 GB EMMC Storage/Chrome OS) C214MA-BU0704...,16990,31990.0,46% off,NaN
2,ASUS Vivobook 15 Core i3 11th Gen - (8 GB/512 GB SSD/Windows 11 Home) X515EA-EJ322WS Thin and Li...,38990,50990.0,23% off,NaN
3,ASUS Vivobook 15 Core i5 11th Gen - (8 GB/512 GB SSD/Windows 11 Home) X515EA-EJ522WS Thin and Li...,47990,69990.0,31% off,NaN
4,HP 14s Intel Core i3 11th Gen - (8 GB/512 GB SSD/Windows 11 Home) 14s - dy2508TU Thin and Light ...,40490,49509.0,18% off,NaN
...,...,...,...,...,...
979,Infinix INBook X1 Neo Series Celeron Quad Core - (4 GB/128 GB SSD/Windows 11 Home) XL22 Thin and...,22990,29990.0,23% off,NaN
980,ASUS Ryzen 7 Quad Core - (16 GB/512 GB SSD/Windows 11 Home) M3504YA-LK541WS Laptop,67210,89900.0,25% off,NaN
981,HP Core i5 13th Gen - (8 GB/512 GB SSD/Windows 11 Home) 15-fd0011TU Thin and Light Laptop,63499,72780.0,12% off,NaN
982,acer Swift 3 Core i5 8th Gen - (8 GB/512 GB SSD/Windows 10 Home/2 GB Graphics) SF314-55G Thin an...,54990,62589.0,12% off,NaN


In [17]:
new_df.drop(columns=['Rating'], axis=1, inplace=True)
new_df.head()

,Product,₹ Current Price,₹ Price Before,Discount
0,Infinix INBook Y1 Plus Intel Core i3 10th Gen - (8 GB/256 GB SSD/Windows 11 Home) XL28 Thin and ...,27990,44990.0,37% off
1,ASUS Chromebook Flip Touch Celeron Dual Core - (4 GB/32 GB EMMC Storage/Chrome OS) C214MA-BU0704...,16990,31990.0,46% off
2,ASUS Vivobook 15 Core i3 11th Gen - (8 GB/512 GB SSD/Windows 11 Home) X515EA-EJ322WS Thin and Li...,38990,50990.0,23% off
3,ASUS Vivobook 15 Core i5 11th Gen - (8 GB/512 GB SSD/Windows 11 Home) X515EA-EJ522WS Thin and Li...,47990,69990.0,31% off
4,HP 14s Intel Core i3 11th Gen - (8 GB/512 GB SSD/Windows 11 Home) 14s - dy2508TU Thin and Light ...,40490,49509.0,18% off


In [18]:
i5i7_laptop = new_df[new_df['Product'].str.contains('|'.join(['i5', 'i7']))]
# new_df[new_df['Product'].str.contains('i5') | new_df['Product'].str.contains('i7')]
laptop_16gb = i5i7_laptop[i5i7_laptop['Product'].str.contains('16 GB')]
nvidia_graphics = laptop_16gb[laptop_16gb['Product'].str.contains('NVIDIA')]
nvidia_graphics

,Product,₹ Current Price,₹ Price Before,Discount
61,acer Predator Helios Neo 16 Core i7 13th Gen - (16 GB/1 TB SSD/Windows 11 Home/8 GB Graphics/NVI...,149990,164999.0,9% off
82,MSI Sword 15 Core i5 12th Gen - (16 GB/512 GB SSD/Windows 11 Home/4 GB Graphics/NVIDIA GeForce R...,85990,98390.0,12% off
116,GIGABYTE G5 KD Core i5 11th Gen - (16 GB/512 GB SSD/Windows 11 Home/6 GB Graphics/NVIDIA GeForce...,89990,151800.0,40% off
146,MSI Core i7 12th Gen - (16 GB/512 GB SSD/Windows 11 Home/6 GB Graphics/NVIDIA GeForce RTX 4050) ...,102990,115990.0,11% off
149,DELL G15 Core i7 12th Gen - (16 GB/1 TB SSD/Windows 11 Home/6 GB Graphics/NVIDIA GeForce RTX 306...,138900,189620.0,26% off
164,MSI Core i7 12th Gen - (16 GB/1 TB SSD/Windows 11 Home/8 GB Graphics/NVIDIA GeForce RTX 3070 Ti/...,165990,227990.0,27% off
189,ASUS ROG Flow Z13 (2022) Core i7 12th Gen - (16 GB/512 GB SSD/Windows 11 Home/4 GB Graphics/NVID...,131049,215990.0,39% off
248,DELL Core i7 11th Gen - (16 GB/512 GB SSD/Windows 11 Home/4 GB Graphics/NVIDIA GeForce RTX 3050T...,99990,128665.0,22% off
264,MSI Core i7 13th Gen - (16 GB/512 GB SSD/Windows 11 Home/6 GB Graphics/NVIDIA GeForce RTX 3050) ...,99990,123990.0,19% off
326,acer Predator Helios Neo 16 Core i7 13th Gen - (16 GB/512 GB SSD/Windows 11 Home/6 GB Graphics/N...,124990,156999.0,20% off


In [19]:
# nvidia_graphics['Current Price'].astype(int)

In [20]:
asus_ng = nvidia_graphics[nvidia_graphics['Product'].str.contains('ASUS')]
asus_ng
# nvidia_graphics[(nvidia_graphics['₹ Current Price'] > 70000)]

,Product,₹ Current Price,₹ Price Before,Discount
189,ASUS ROG Flow Z13 (2022) Core i7 12th Gen - (16 GB/512 GB SSD/Windows 11 Home/4 GB Graphics/NVID...,131049,215990.0,39% off
466,ASUS TUF Dash F15 Core i5 12th Gen - (16 GB/512 GB SSD/Windows 11 Home/4 GB Graphics/NVIDIA GeFo...,83990,122990.0,31% off
614,ASUS Core i5 12th Gen - (16 GB/512 GB SSD/Windows 11 Home/4 GB Graphics/NVIDIA GeForce RTX RTX 3...,92990,114990.0,19% off
642,ASUS TUF Gaming F17 (2022) Core i7 12th Gen - (16 GB/1 TB SSD/Windows 11 Home/4 GB Graphics/NVID...,114990,147990.0,22% off
700,ASUS Vivobook Pro 15 OLED Core i5 12th Gen - (16 GB/512 GB SSD/Windows 11 Home/4 GB Graphics/NVI...,89990,110990.0,18% off
717,ASUS TUF Dash F15 Core i7 12th Gen - (16 GB/1 TB SSD/Windows 11 Home/4 GB Graphics/NVIDIA GeForc...,103990,120990.0,14% off
753,ASUS Vivobook Pro 15 OLED Core i5 12th Gen - (16 GB/512 GB SSD/Windows 11 Home/4 GB Graphics/NVI...,89990,110990.0,18% off
784,ASUS Core i7 12th Gen - (16 GB/1 TB SSD/Windows 11 Home/6 GB Graphics/NVIDIA GeForce RTX RTX3060...,143918,170990.0,15% off
906,ASUS Core i7 12th Gen - (16 GB/1 TB SSD/Windows 11 Home/8 GB Graphics/NVIDIA GeForce RTX RTX 307...,132982,188990.0,29% off
